# Lagrangian and Eulerian demonstration on ECCO

`seaduck` Lagrangian particle demonstration. This version uses the ECCO MITgcm velocity field data available on SciServer.

authors: Wenrui Jiang, Tom Haine Feb '23

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np

import seaduck as sd

### The ECCO MITgcm run is a low-resolution global state estimate, available on SciServer. The simulation is opened using the OceanSpy package.

Choose between the monthly-mean data ('ECCO') or the daily-mean data ('daily_ecco').

### See: https://dev-poseidon-ocean.pantheonsite.io/products/datasets/.

In [ ]:
# ecco = ospy.open_oceandataset.from_catalog("ECCO")
# ecco = ospy.open_oceandataset.from_catalog('daily_ecco')
ecco = sd.utils.get_dataset("ecco")
ecco = sd.utils.process_ecco(ecco)

In [ ]:
ecco

### Initialize seaduck package. Then set the debug level.

In [ ]:
sd.rcParam[
    "debug_level"
] = "very_high"  # Options are: 'high','very_high','not that high'

### Specify the parameters for the particles (number, positions, start time).

In [ ]:
# Define the extend of the box
west = -19.0
east = -9.0
south = 57.0
north = 63.0
shallow = -10.0
deep = -10.0

time = "1992-02-15"

Nlon = 20  # How many along longitudinal direction?
Nlat = 20  # How many along latitudinal direction?
Ndep = 1  # How many along vertical direction?

x, y, z, t = sd.utils.easy_3d_cube(
    (east, west, Nlon),
    (south, north, Nlat),
    (shallow, deep, Ndep),
    time,
    print_total_number=True,
)

In [ ]:
# x,y,z,t =
# # Change the number of particles here
# N = int(1e2)

# # Change the vertical depth of the particles here
# levels = np.array([-5])
# sqrtN = int(np.sqrt(N))

# # Change the longitude and latitude positions of the particles here
# xx = np.linspace(-19, -9, sqrtN)
# yy = np.linspace(63, 57, sqrtN)

# # Compute intermediate grid variables
# xxx, yyy = np.meshgrid(xx, yy)
# x = xxx.ravel()
# y = yyy.ravel()
# x, z = np.meshgrid(x, levels)
# y, z = np.meshgrid(y, levels)
# x = x.ravel()
# y = y.ravel()
# z = z.ravel()

# # Change the times here
# start_time = "1998-12-15"
# t = (
#     np.array([np.datetime64(start_time) for i in x]) - np.datetime64("1970-01-01")
# ) / np.timedelta64(1, "s")

#### Plot the particle positions

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.plot(x, y, "r+")
ax.coastlines()
ax.set_xlim([-25, 0])
ax.set_ylim([50, 70])
plt.show()

### Let's explore the interpolation function in `seaduck`:

In [ ]:
help(sd.OceInterp)

### Interpolate these ECCO fields onto the particle positions.

In [ ]:
[s, (u, v), eta, mask] = sd.OceInterp(
    ecco, ["SALT", ("UVELMASS", "VVELMASS"), "ETAN", "maskC"], x, y, z, t
)

### Plot the interpolated salinity field.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
c = ax.scatter(x, y, c=s)
ax.coastlines()
ax.set_xlim([-25, 0])
ax.set_ylim([50, 70])
plt.colorbar(c)
plt.title("Salinity (psu)")

### Plot the interpolated $u$ field.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
c = ax.scatter(x, y, c=u)
ax.coastlines()
ax.set_xlim([-25, 0])
ax.set_ylim([50, 70])
plt.colorbar(c)
plt.title("Zonal Velocity (m/s)")

### Plot the interpolated $\\eta$ field.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
c = ax.scatter(x, y, c=eta)
ax.coastlines()
ax.set_xlim([-25, 0])
ax.set_ylim([50, 70])
plt.colorbar(c)
plt.title("Sea Surface Height (m)")

### Now compute Lagrangian trajectories for these particles.

#### Define the `start_time` and `end_time`. Here the particles are integrated backwards in time.

In [ ]:
start_time = "1992-01-17"
end_time = "1992-03-12"

t_bnds = np.array(
    [
        np.datetime64(start_time) - np.datetime64("1970-01-01"),
        np.datetime64(end_time) - np.datetime64("1970-01-01"),
    ]
) / np.timedelta64(1, "s")

### Perform the particle trajectory simulation.

#### Record the salinity along the particle trajectories as well as their (lat,lon) positions.

In [ ]:
stops, [s, raw, lat, lon] = sd.OceInterp(
    ecco,
    ["SALT", "__particle.raw", "__particle.lat", "__particle.lon"],
    x,
    y,
    z,
    t_bnds,
    lagrangian=True,
    return_pt_time=True,
    lagrange_kwarg={"save_raw": True},
)

#### There are 4 output times. See also the diagnostic output from running the integration.

In [ ]:
stops

#### The `raw` output is a vector of `OceInterp` objects with position information

In [ ]:
raw

### Plot the interpolated salinity field on the final particle positions.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.scatter(lon[-1], lat[-1], c=s[-1], s=1)
ax.coastlines()
ax.set_xlim([-70, 0])
ax.set_ylim([30, 70])
plt.title("salinity map")
plt.show()

### There are some other `OceInterp` classes:

#### 1. Kernel object

In [ ]:
KnW = sd.kernel_weight.KnW

#### Define derivative kernels for

$\\partial / \\partial z$, $\\partial^2 / \\partial x^2$, and $\\partial / \\partial t$.

In [ ]:
default = KnW()
dz_kernel = KnW(vkernel="dz")
dx2_kernel = KnW(
    hkernel="dx", h_order=2, inheritance=[[0, 1, 2, 3, 4, 5, 6, 7, 8]], tkernel="linear"
)
dt_kernel = KnW(tkernel="dt")

#### Apply the kernels to the ECCO fields:

In [ ]:
[dsdz, (d2udx2, dvdt)] = sd.OceInterp(
    ecco,
    {"SALT": dz_kernel, ("UVELMASS", "VVELMASS"): (dx2_kernel, dt_kernel)},
    x,
    y,
    z,
    t,
)

### Plot one of the differentiated fields on the initial particle positions.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
c = ax.scatter(x, y, c=d2udx2)
ax.coastlines()
ax.set_xlim([-25, 0])
ax.set_ylim([50, 70])
plt.colorbar(c)
plt.title("Second Derivative of the Zonal Velocity (m/s per grid scale squared)")
plt.show()